In [1]:
'''
/* ----------------------------------------------------------------------------
 * Copyright 2023, CURLY Lab, University of Michigan
 * All Rights Reserved
 * See LICENSE for the license information
 * -------------------------------------------------------------------------- */

/**
 *  @file   run_neya_MEKF.py
 *  @author Wenzhe Tong
 *  @brief  Multiplicative EKF
 *  @date   August 4, 2023
 **/
'''

from MEKF import *
import os

In [5]:
root_path = "/media/justin/DATA/data/neya/"
directory_name = ["paintball", "pelenor"]
directory_names = [root_path+_ for _ in directory_name]

topic_name = '/mrzr/localization/piksi_attitude/navsatfix_spp'
acc_cov = 0.1 # check in datasheet
gyro_cov = 0.01 # check in datasheet
wheel_encoder_cov = 0.5 # check in rosbag

for dir_name in directory_names:
    for filename in os.listdir(dir_name):
        if filename.endswith(".bag"):
            short_filename = filename.split("_general")[0]
            print("Processing: "+short_filename+" "+dir_name+"/"+filename)
            if not os.path.exists(root_path+"/result/"+short_filename):
                os.makedirs(root_path+"/result/"+short_filename)
            bag = rosbag.Bag(dir_name+"/"+filename)
            
            ################### Initialization #####################
            ##### Neya #####
            imu_data = read_imu(bag, "/mrzr/localization/imu/imu") # neya
            vel_data = read_vel(bag, "/mrzr/localization/wheel_encoder/twist") # neya
            # R_imu2body = quat2rot([1., 0., 0., 0.]) # neya #TODO: need to check
            R_imu2body = quat2rot([0.7071068, 0, 0, 0.7071068])
            # TODO: add bias initialization
            # acc_bias = np.array([0.01,   -0.0642287,    -0.155932]).reshape(3)
            # gyro_bias = np.array([-7.49327e-05,  0.000315891,   0.00032303]).reshape(3)

            gyro_bias, acc_bias= init_bias(imu_data, R_imu2body,250)
            print("bias_initialized as: ",gyro_bias, acc_bias)

            t_imu = imu_data[:, 0]
            gyro_data = imu_data[:, 1:4]
            acc_data = imu_data[:, 4:7]
            
            t_vel = vel_data[:, 0]
            vel_data = vel_data[:, 1:4]
            # gyro_data = ((gyro_data.T)).T
            # acc_data = ((acc_data.T)).T
            # quat_data = 

            g = np.array([0, 0, 9.81]).reshape(3)

            # init state
            # X = [p, v, q]
            p_est = np.zeros([imu_data.shape[0], 3]) # position estimates
            v_est = np.zeros([imu_data.shape[0], 3]) # velocity estimates
            q_est = np.zeros([imu_data.shape[0], 4]) # quaternion estimates
            p_cov = np.zeros([imu_data.shape[0], 9, 9]) # state covariance

            p_est[0] = np.array([0, 0, 0])
            v_est[0] = np.array([0, 0, 0])
            q_est[0] = np.array([1, 0, 0, 0])
            p_cov[0] = np.zeros([9, 9])
            
            
            ################## Main loop ################            
            i = 0
            for k in tqdm(range(1, t_imu.shape[0])): # start at 1 because we have initial state
                # Time difference
                dt = t_imu[k] - t_imu[k-1]
                
                # 1. Update state with IMU inputs
                R_ns = quat2rot(q_est[k-1, :])
                f_ns = (R_ns @ (R_imu2body @ acc_data[k]-acc_bias).reshape((3,1))) - g.reshape((3,1)) # acc_bias in body frame
                assert f_ns.shape == (3,1)
                
                p_ = p_est[k-1, :].reshape((3,1)) + (v_est[k-1, :]*dt).reshape((3,1)) + 0.5*f_ns*dt**2 
                v_ = v_est[k-1, :].reshape((3,1)) + f_ns*dt
                R_predict = R_ns @ expm(skew_symmetric((R_imu2body@gyro_data[k]-gyro_bias)*dt))
                q_ = rot2quat(R_predict).reshape((4,1))
                
                
                zero = np.zeros((3,3))
                I = np.eye(3)
                F = expm( np.block([[ zero ,   I,                                            zero],
                                    [ zero ,   zero, -R_predict @ skew_symmetric(R_imu2body@acc_data[k] - acc_bias)],
                                    [ zero ,   zero,      -skew_symmetric(R_imu2body@gyro_data[k]-gyro_bias)]])*dt )

                q_cov = np.zeros([9, 9]) # IMU noise covariance
                q_cov[0:3, 0:3] = 0.000001*np.eye(3)
                q_cov[3:6, 3:6] = acc_cov * np.eye(3) *dt**2
                q_cov[6:9, 6:9] = gyro_cov * np.eye(3) *dt**2
                
                l_jac = np.zeros((9,6))
                l_jac[3:, :] = np.eye(6) # motion model noise jacobian
                p_cov_ = F @ p_cov[k-1, :, :] @ F.T + q_cov 
                
                if i<vel_data.shape[0] and t_imu[k] >= t_vel[i]:
                    p_, v_, q_, p_cov_ = measurement_update(wheel_encoder_cov, vel_data[i, :].reshape((3,1)), p_cov_, p_, v_, q_)
                    i+=1
                
                # 1.4 update states
                p_est[k, :] = p_.T
                v_est[k, :] = v_.T
                q_est[k, :] = q_.T
                p_cov[k, :, :] = p_cov_.T
            
            save_pose_to_tum(t_imu, p_est, q_est, filename=root_path+"/result/"+short_filename+"/mekf.txt")


Processing: follow /media/justin/DATA/data/neya/paintball/follow_general_2022-12-01-15-44-43_0.bag
imu data shape:  (63357, 11)
imu data frequency:  198.93750879068372
vel data shape:  (15933, 4)
vel data frequency:  50.002029287545476
[-0.14576548 -0.06385911  9.62389437]
[-0.14576548 -0.06385911 -0.18610563]
bias_initialized as:  [ 1.57418214e-05 -1.09695648e-04 -7.28956014e-04] [ 0.06385911 -0.14576548 -0.18610563]


100%|██████████| 63356/63356 [00:20<00:00, 3164.27it/s]


Processing: hill_loop_dynamic /media/justin/DATA/data/neya/paintball/hill_loop_dynamic_general_2022-12-01-16-10-10_0.bag
imu data shape:  (26814, 11)
imu data frequency:  198.9382871799949
vel data shape:  (6748, 4)
vel data frequency:  50.01676274423788
[-3.79159288e-01 -8.90489238e-03  9.61865942e+00]
[-0.37915929 -0.00890489 -0.19134058]
bias_initialized as:  [ 1.12785021e-04 -5.49566923e-05 -1.22364813e-04] [ 0.00890489 -0.37915929 -0.19134058]


100%|██████████| 26813/26813 [00:08<00:00, 3131.30it/s]


Processing: hill_loop /media/justin/DATA/data/neya/paintball/hill_loop_general_2022-12-01-15-30-02_0.bag
imu data shape:  (32037, 11)
imu data frequency:  198.9129307888899
vel data shape:  (8059, 4)
vel data frequency:  50.00000130482249
[0.34366544 0.21397998 9.57083463]
[ 0.34366544  0.21397998 -0.23916537]
bias_initialized as:  [4.32717782e-05 3.81288757e-05 2.97205418e-05] [-0.21397998  0.34366544 -0.23916537]


...

100%|██████████| 29977/29977 [00:08<00:00, 3582.45it/s]


Processing: high_speed_loop /media/justin/DATA/data/neya/pelenor/high_speed_loop_general_2022-12-01-10-12-17_0.bag
imu data shape:  (14259, 11)
imu data frequency:  198.93816114318054
vel data shape:  (3591, 4)
vel data frequency:  50.01071456184518
[ 0.26460762 -0.06422866  9.64406826]
[ 0.26460762 -0.06422866 -0.16593174]
bias_initialized as:  [-3.15890735e-04 -7.49327042e-05  3.23029884e-04] [ 0.06422866  0.26460762 -0.16593174]


100%|██████████| 14258/14258 [00:04<00:00, 3043.39it/s]


Processing: low_speed_loop /media/justin/DATA/data/neya/pelenor/low_speed_loop_general_2022-12-01-10-09-04_0.bag
imu data shape:  (30368, 11)
imu data frequency:  198.937726553926
vel data shape:  (7641, 4)
vel data frequency:  50.011488797564
[ 0.28728302 -0.37559708  9.64339701]
[ 0.28728302 -0.37559708 -0.16660299]
bias_initialized as:  [ 2.63113528e-07  5.22112921e-05 -1.31969943e-04] [ 0.37559708  0.28728302 -0.16660299]


100%|██████████| 30367/30367 [00:08<00:00, 3395.54it/s]


In [ ]:

i = 0
for k in tqdm(range(1, t_imu.shape[0])): # start at 1 because we have initial state
    # Time difference

    dt = t_imu[k] - t_imu[k-1]
    
    # 1. Update state with IMU inputs
    # q_prev = Quaternion(*q_est[k-1, :])
    # q_curr = Quaternion(axis_angle=(gyro_data[k]-gyro_bias)*dt)
    # R_ns = q_prev.to_mat()
    R_ns = quat2rot(q_est[k-1, :])
    f_ns = (R_ns @ (R_imu2body @ acc_data[k]-acc_bias).reshape((3,1))) - g.reshape((3,1)) # acc_bias in body frame
    # print(R_ns @ (R_imu2body @ acc_data[k]-acc_bias).reshape((3,1)))
    # print(f_ns)
    # print("-------")    
    assert f_ns.shape == (3,1)
    
    p_ = p_est[k-1, :].reshape((3,1)) + (v_est[k-1, :]*dt).reshape((3,1)) + 0.5*f_ns*dt**2 
    v_ = v_est[k-1, :].reshape((3,1)) + f_ns*dt
    # q_ = q_prev.quat_mult_left(q_curr).reshape((4,1))
    R_predict = R_ns @ expm(skew_symmetric((R_imu2body@gyro_data[k]-gyro_bias)*dt))
    # print(gyro_data[k])
    # print("---")
    q_ = rot2quat(R_predict).reshape((4,1))
    
    # 1.1 get motion model Jacobian
    # F = np.eye(9)
    # F[0:3, 3:6] = np.eye(3)*dt
    # F[3:6, 6:9] = -R_ns @ skew_symmetric((acc_data[k]-acc_bias))*dt
    # F[6:9, 6:9] = Quaternion(aq_xis_angle=(gyro_data[k]-gyro_bias)*dt).to_mat().T
    
    zero = np.zeros((3,3))
    I = np.eye(3)
    F = expm( np.block([[ zero ,   I,                                            zero],
                        [ zero ,   zero, -R_predict @ skew_symmetric(R_imu2body@acc_data[k] - acc_bias)],
                        [ zero ,   zero,      -skew_symmetric(R_imu2body@gyro_data[k]-gyro_bias)]])*dt )

    # 1.2 uncertainty propagation
    q_cov = np.zeros([9, 9]) # IMU noise covariance
    q_cov[0:3, 0:3] = 0.000001*np.eye(3)
    q_cov[3:6, 3:6] = acc_cov * np.eye(3) *dt**2
    q_cov[6:9, 6:9] = gyro_cov * np.eye(3) *dt**2
    
    l_jac = np.zeros((9,6))
    l_jac[3:, :] = np.eye(6) # motion model noise jacobian
    p_cov_ = F @ p_cov[k-1, :, :] @ F.T + q_cov 
    
    # # 1.3 velocity update
    # if(k<t_vel.shape[0] and abs(t_imu[k]-t_vel[i]) <= abs(t_imu[k+1]-t_vel[i]) and i<vel_data.shape[0]):
    #     p_, v_, q_, p_cov_ = measurement_update(wheel_encoder_cov, vel_data[i, :].reshape((3,1)), p_cov_, p_, v_, q_)
    #     i+=1
    if i<vel_data.shape[0] and t_imu[k] >= t_vel[i]:
        p_, v_, q_, p_cov_ = measurement_update(wheel_encoder_cov, vel_data[i, :].reshape((3,1)), p_cov_, p_, v_, q_)
        i+=1
    
    # 1.4 update states
    p_est[k, :] = p_.T
    v_est[k, :] = v_.T
    q_est[k, :] = q_.T
    p_cov[k, :, :] = p_cov_.T


In [ ]:
save_pose_to_tum(t_imu, p_est, q_est, filename='tum_estimated.txt')